In [1]:
import numpy as np
import pandas as pd
from glob import glob
import sys
import os

import matplotlib.pyplot as plt

sys.path.append("..") 
from tp_utils import *
from tp_config import *
from tp_assets import *

/home/samsmu/anaconda3/envs/t/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/samsmu/anaconda3/envs/t/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


ModuleNotFoundError: No module named 'tp_config'

In [4]:
TOKEN0 = 't.UFRJ8SC9hafVOhFxEUY7yf1wZ1gGhwJp-WCp9o4rnEChHWns0c3jQ21eQwoOW_RurFqeZpss2scJkmMQnomJ9g'
TOKEN1 = 't.6nHltT1dYSfrVTIV9zF72fxDlB2sXJbRD6iJNpZXTFAN61rmD7m71xPp9ko12ta1JxA06em4YdN36xicnBmjWg'

token = TOKEN0

## Tinkoff sandbox

In [12]:
logging.basicConfig(format="%(asctime)s %(levelname)s:%(message)s", level=logging.DEBUG)
logger = logging.getLogger(__name__)


def add_money_sandbox(client, account_id, money, currency="rub"):
    """Function to add money to sandbox account."""
    money = decimal_to_quotation(Decimal(money))
    return client.sandbox.sandbox_pay_in(
        account_id=account_id,
        amount=MoneyValue(units=money.units, nano=money.nano, currency=currency),
    )

### Get accounts

In [8]:
"""Example - How to set/get balance for sandbox account.
How to get/close all sandbox accounts.
How to open new sandbox account."""
with SandboxClient(token) as client:
    # get all sandbox accounts
    sandbox_accounts = client.users.get_accounts()
    print(sandbox_accounts)

    # close all sandbox accounts
    for sandbox_account in sandbox_accounts.accounts:
        client.sandbox.close_sandbox_account(account_id=sandbox_account.id)

    # open new sandbox account
    sandbox_account = client.sandbox.open_sandbox_account()
    print(sandbox_account.account_id)

    account_id = sandbox_account.account_id

GetAccountsResponse(accounts=[Account(id='906e2901-c68e-4848-944c-9617aadd216d', type=<AccountType.ACCOUNT_TYPE_TINKOFF: 1>, name='', status=<AccountStatus.ACCOUNT_STATUS_OPEN: 2>, opened_date=datetime.datetime(2024, 1, 18, 12, 49, 28, 599230, tzinfo=datetime.timezone.utc), closed_date=datetime.datetime(1970, 1, 1, 0, 0, tzinfo=datetime.timezone.utc), access_level=<AccessLevel.ACCOUNT_ACCESS_LEVEL_FULL_ACCESS: 1>)])
064522ee-7760-422b-8995-1d1a4476c83b


### Add money to Account

In [13]:
with SandboxClient(token) as client:
      # add initial 2 000 000 to sandbox account
        print(add_money_sandbox(client=client, account_id=account_id, money=2000000))
        logger.info(
            "positions: %s", client.operations.get_positions(account_id=account_id)
        )
        print(
            "money: ",
            float(
                quotation_to_decimal(
                    client.operations.get_positions(account_id=account_id).money[0]
                )
            ),
        )

2024-03-31 11:21:11,289 INFO:0e342b0c6c227b74cb7cd866751aa2cf SandboxPayIn
2024-03-31 11:21:11,328 INFO:52731d154e4ab6a4a2f96b16b26ad1e3 GetPositions
2024-03-31 11:21:11,329 INFO:positions: PositionsResponse(money=[MoneyValue(currency='rub', units=2000000, nano=0)], blocked=[MoneyValue(currency='rub', units=0, nano=0)], securities=[], limits_loading_in_progress=False, futures=[], options=[])
2024-03-31 11:21:11,362 INFO:dd96ccec7d690375d4d62bb393b9a9a4 GetPositions


SandboxPayInResponse(balance=MoneyValue(currency='rub', units=2000000, nano=0))
money:  2000000.0


### Account Operations

In [14]:
with SandboxClient(token) as client:
    logger.info("orders: %s", client.orders.get_orders(account_id=account_id))
    logger.info(
        "positions: %s", client.operations.get_positions(account_id=account_id)
    )
    logger.info(
        "portfolio: %s", client.operations.get_portfolio(account_id=account_id)
    )
    logger.info(
        "operations: %s",
        client.operations.get_operations(
            account_id=account_id,
            from_=datetime(2023, 1, 1),
            to=datetime(2023, 2, 5),
        ),
    )
    logger.info(
        "withdraw_limits: %s",
        client.operations.get_withdraw_limits(account_id=account_id),
    )

2024-03-31 11:23:15,354 INFO:e5ace91b692eae9d8495b714896f3d8d GetOrders
2024-03-31 11:23:15,355 INFO:orders: GetOrdersResponse(orders=[])
2024-03-31 11:23:15,406 INFO:6e70d944f04fb9909701df51aad9c7b6 GetPositions
2024-03-31 11:23:15,406 INFO:positions: PositionsResponse(money=[MoneyValue(currency='rub', units=2000000, nano=0)], blocked=[MoneyValue(currency='rub', units=0, nano=0)], securities=[], limits_loading_in_progress=False, futures=[], options=[])
2024-03-31 11:23:15,442 INFO:74ae5ded9b5919aa893f935a8d5ae613 GetPortfolio
2024-03-31 11:23:15,444 INFO:portfolio: PortfolioResponse(total_amount_shares=MoneyValue(currency='rub', units=0, nano=0), total_amount_bonds=MoneyValue(currency='rub', units=0, nano=0), total_amount_etf=MoneyValue(currency='rub', units=0, nano=0), total_amount_currencies=MoneyValue(currency='rub', units=2000000, nano=0), total_amount_futures=MoneyValue(currency='rub', units=0, nano=0), expected_yield=Quotation(units=0, nano=0), positions=[PortfolioPosition(figi=

### Закрытие аккаунта

In [15]:
with SandboxClient(token) as client:        
        # add + 2 000 000 to sandbox account, total is 4 000 000
        print(add_money_sandbox(client=client, account_id=account_id, money=2000000))
        logger.info(
            "positions: %s", client.operations.get_positions(account_id=account_id)
        )

        # close new sandbox account
        sandbox_account = client.sandbox.close_sandbox_account(
            account_id=sandbox_account.account_id
        )
        print(sandbox_account)


2024-03-31 11:25:02,638 INFO:479c3b336f5f9bc9e1461d06abae047a SandboxPayIn
2024-03-31 11:25:02,674 INFO:6b6c36a8dfbc7198565f34064a34f626 GetPositions
2024-03-31 11:25:02,675 INFO:positions: PositionsResponse(money=[MoneyValue(currency='rub', units=4000000, nano=0)], blocked=[MoneyValue(currency='rub', units=0, nano=0)], securities=[], limits_loading_in_progress=False, futures=[], options=[])
2024-03-31 11:25:02,719 INFO:2467bc738855363080bc5e2f5e9f6a5a CloseSandboxAccount


SandboxPayInResponse(balance=MoneyValue(currency='rub', units=4000000, nano=0))
CloseSandboxAccountResponse()


## Helper functions

In [2]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

from datetime import datetime
now = datetime.today().strftime('%Y-%m-%d')


In [16]:
def calculate_portfolio_difference(old_portfolio, new_portfolio):
    """
    Рассчитать разницу между двумя портфелями.

    Args:
        old_portfolio: Словарь, где ключом является тикер, а значением - количество акций.
        new_portfolio: Словарь, где ключом является тикер, а значением - количество акций.

    Returns:
        Словарь, где ключом является тикер, а значением - разница между количеством акций в 
        новых и старых портфелях.
    """

    difference = {}
    for ticker in new_portfolio:
        if ticker in old_portfolio:
            difference[ticker] = new_portfolio[ticker] - old_portfolio[ticker]
        else:
            difference[ticker] = new_portfolio[ticker]

    for ticker in old_portfolio:
        if ticker not in new_portfolio:
            difference[ticker] = -old_portfolio[ticker]

    return difference


In [31]:
def calculate_portfolio_allocation(weights, total_amount, lot_sizes):
    """
    Рассчитать распределение суммы на веса в портфеле, учитывая лоты.

    Args:
        weights: Список весов акций в портфеле.
        total_amount: Общая сумма, которую нужно распределить.
        lot_sizes: Список размеров лотов для акций.

    Returns:
        Список, где каждый элемент - количество акций, 
        округленное до размера лота.
    """

    # Нормализация весов
    weights = normalize_weights(weights)

    # Распределение суммы
    amounts = [weight * total_amount for weight in weights]

    # Округление до размера лота
    quantities = [round(amount / lot_size, 0) for amount, lot_size in zip(amounts, lot_sizes)]

    # Корректировка с учетом минимального лота
    for i in range(len(quantities)):
        if lot_sizes[i] > 0 and quantities[i] * lot_sizes[i] < total_amount:
            quantities[i] = 1

    # Распределение остатка
    remaining_amount = total_amount - sum(quantity * lot_size for quantity, lot_size in zip(quantities, lot_sizes))

    return quantities, remaining_amount


def normalize_weights(weights):
    """
    Нормализовать веса, чтобы они давали в сумме единицу.

    Args:
        weights: Список весов.

    Returns:
        Список нормализованных весов.
    """

    total_weight = sum(weights)
    return [weight / total_weight for weight in weights]


# Пример
weights = [0.4, 0.3, 0.3]
total_amount = 10000
lot_sizes = [1, 10, 1000]

quantities, rem = calculate_portfolio_allocation(weights, total_amount, lot_sizes)

# Вывод
for quantity, weight in zip(quantities, weights):
    print(f"Количество: {quantity}, Вес: {weight:.2%}")

print(f"\nОбщая сумма: {sum(quantity * lot_size for quantity, lot_size in zip(quantities, lot_sizes))}")

Количество: 1, Вес: 40.00%
Количество: 1, Вес: 30.00%
Количество: 1, Вес: 30.00%

Общая сумма: 1011


In [21]:
def calculate_portfolio_allocation(weights, total_amount, lot_sizes):
    """
    Рассчитать распределение суммы на веса в портфеле, учитывая лоты.

    Args:
        weights: Список весов акций в портфеле.
        total_amount: Общая сумма, которую нужно распределить.
        lot_sizes: Список размеров лотов для акций.

    Returns:
        Список, где каждый элемент - 
        словарь с информацией о количестве акций, цене за 1 акцию, 
        цене за лот и стоимости позиции.
    """

    # Нормализация весов
    weights = normalize_weights(weights)

    # Распределение суммы
    amounts = [weight * total_amount for weight in weights]

    # Округление до размера лота
    quantities = [round_to_lot_size(amount, lot_size) for amount, lot_size in zip(amounts, lot_sizes)]

    # Расчет стоимости позиции
    position_values = [quantity * lot_size for quantity, lot_size in zip(quantities, lot_sizes)]

    # Формирование выходных данных
    portfolio = []
    for quantity, lot_size, position_value in zip(quantities, lot_sizes, position_values):
        portfolio.append({
            "quantity": quantity,
            "price_per_lot": position_value / quantity,
            "position_value": position_value,
        })

    return portfolio


def round_to_lot_size(amount, lot_size):
    """
    Округлить число до размера лота.

    Args:
        amount: Число, которое нужно округлить.
        lot_size: Размер лота.

    Returns:
        Округленное число.
    """

    if lot_size == 1:
        return round(amount)

    return round(amount / lot_size) * lot_size


# Пример
weights = [0.4, 0.3, 0.3]
total_amount = 10000
lot_sizes = [1, 10, 1000]

portfolio = calculate_portfolio_allocation(weights, total_amount, lot_sizes)

# Вывод
for position in portfolio:
    print(f"Количество: {position['quantity']}")
    print(f"Цена за лот: {position['price_per_lot']}")
    print(f"Стоимость позиции: {position['position_value']}")
    print()

print(f"\nОбщая сумма: {sum(position['position_value'] for position in portfolio)}")

Количество: 4000
Цена за лот: 1.0
Стоимость позиции: 4000

Количество: 3000
Цена за лот: 10.0
Стоимость позиции: 30000

Количество: 3000
Цена за лот: 1000.0
Стоимость позиции: 3000000


Общая сумма: 3034000
